<a href="https://colab.research.google.com/github/lokeshparab/Stock-Market-Prediction-Analysis-Forecasting/blob/main/Stock_Market_Prediction_Analysis_Forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
# !pip install yfinance --upgrade --no-cache-dir
# !pip install Yahoo-ticker-downloader

# Data Collection Phase

## Import Library

In [1]:
import yfinance as yf
import requests as rq
import json     as js
import os

from bs4      import BeautifulSoup as BS
from datetime import datetime      as dt
from tqdm     import tqdm

## Scrape Latest Trending Tickers in Yahoo finance

In [2]:
url = "https://finance.yahoo.com/lookup/"
FOLDER_PATH = 'Trend Ticker List'

# get url request data
res = rq.get(url)

# If response status is not ok
if not res.ok:
  print(res)

# Scrape ticker symbol from website
else:
  soup             = BS(res.content,'html.parser')
  ticker_list_info = soup.find('section', id = 'lookup-page').find('tbody').findAll('tr')
  ticker_symbols    = [ sym.find('a').text for sym in ticker_list_info ]

  current_time     = dt.today().strftime("%d-%m-%Y %H:%M:%S")

  # Saving datetime and ticker list in dictionary
  data = {
      'datetime': current_time,
      'ticker_list' : ticker_symbols
  }


  os.makedirs(FOLDER_PATH, exist_ok = True)

  # Saving ticker symbol data in json file at folder "Trend Ticker List"
  with open(f'{FOLDER_PATH}/{current_time}.json','w') as fd:
    fd.write( js.dumps(data) )
    fd.close()

In [3]:

#@markdown ## Get History data and metadata from Trend Ticker List
#@markdown Optional `file_name` attribute if you want specific files to extract ticker list otherwise is it taken `lastest` by default
file_name = 'latest' #@param{type:"string"}
duration = '1y' #@param['1y','2y','4y','6y','1m','2m','4m','6m']

FOLDER_PATH = 'Trend Ticker List'

# Read file function for json
def read_file(path:str)->dict:
  with open(path,'r') as fd:
    data:dict = js.loads(fd.read())
    fd.close()
    return data

# Get sorted file from Folder "Trend Ticker List"
files = sorted(os.listdir(FOLDER_PATH))

# Generate file path for existing file or lastest file
FILE_PATH  :str  = f'{FOLDER_PATH}/' + ( file_name if file_name in files else files[-1] )

# Get ticket data from generated file path
ticker_data:dict = read_file(path = FILE_PATH)

# Generate Folder path for History data
FOLDER_PATH = f"History data ending from {ticker_data['datetime'].split()[0]} for {duration}"
os.makedirs(FOLDER_PATH, exist_ok = True)

# Create metadata info for all ticker symbol
metadata = {
    'duration' : duration ,
    'tickers'  : []
    }

# Get metadata and save history data in CSV file at folder "History duration .."
for ticker in tqdm(ticker_data['ticker_list']):

  msft = yf.Ticker(ticker)

  metadata['tickers'].append(msft.info)

  msft.history(period=duration).reset_index().to_csv(f'{FOLDER_PATH}/{ticker}.csv')

# Save metadata
with open(f'{FOLDER_PATH}/metadata.json','w') as fd:
  fd.write(js.dumps(metadata))
  fd.close()


100%|██████████| 30/30 [00:23<00:00,  1.26it/s]


In [6]:
!zip -r "$FOLDER_PATH" "$FOLDER_PATH"
!zip -r 'Trend Ticker List' 'Trend Ticker List'

	zip warning: name not matched: $FOLDER_PATH

zip error: Nothing to do! (try: zip -r $FOLDER_PATH . -i $FOLDER_PATH)
updating: Trend Ticker List/ (stored 0%)
updating: Trend Ticker List/11-10-2023 14:13:31.json (deflated 36%)


# Data Analysis Phase